In [1]:
from cgisim_sims import cgisim_sims
import matplotlib.pylab as plt
import proper
import roman_phasec_proper
import numpy as np
import astropy.io.fits as pyfits
from scipy.ndimage import rotate

In [2]:
cgisim_obj = cgisim_sims()

# Set DM solutions
dm1 = proper.prop_fits_read( roman_phasec_proper.lib_dir + '/examples/hlc_best_contrast_dm1.fits')
dm2 = proper.prop_fits_read( roman_phasec_proper.lib_dir + '/examples/hlc_best_contrast_dm2.fits')
cgisim_obj.options['dm1'] = dm1
cgisim_obj.options['dm2'] = dm2

In [3]:
name_scene = 'example_pointsourcecompanions_rollsandRefv7'
flag_use_emccd = False

In [4]:
# Science target star
star_vmag = 5.04
cgisim_obj.sources[0]['star_vmag'] = star_vmag
cgisim_obj.sources[0]['star_type'] = 'g0v'
cgisim_obj.sources[0]['name'] = '47Uma'

# Reference star
starref_vmag = 5.04
cgisim_obj.sources[1]['star_vmag'] = starref_vmag
cgisim_obj.sources[1]['star_type'] = 'g0v'
cgisim_obj.sources[1]['name'] = 'rPup'

# Planet
starref_vmag = star_vmag + 8*2.5
cgisim_obj.sources[2]['star_vmag'] = starref_vmag
cgisim_obj.sources[2]['star_type'] = 'g0v'
cgisim_obj.sources[2]['name'] = 'example'

In [5]:
# Read in LOWFE
datadir_Z411 = "/Users/sammyh/Codes/cgisim_sims/data/hlc_os11_v2/" # "../cgisim_sims/data/hlc_os11_v2/"
flnm_Z411 = "hlc_os11_inputs.fits"
inFile = pyfits.open(datadir_Z411+flnm_Z411)
hlc_os11_inputs = inFile[0].data

# Retrieve jitter values
jitt_sig_x_arr = hlc_os11_inputs[:,78] * 1 # [masRMS]
jitt_sig_y_arr = hlc_os11_inputs[:,79] * 1 # [masRMS]

# Retrieve Z4-Z11 values
z411_mat = hlc_os11_inputs[:,46:54]

In [6]:
# Create scene
# Batch IDs -- for naming
batch_id_os11 = hlc_os11_inputs[:,2]

# Create scene with LO errors
cgisim_obj.generate_scene(name=name_scene,jitter_x=jitt_sig_x_arr,jitter_y=jitt_sig_y_arr,
                          zindex=np.arange(4,11+1),zval_m=z411_mat)

# Initialize schedule_index_array
cgisim_obj.scene['schedule']['schedule_index_array'] = []

In [7]:
# Observe reference target
index_batch_ref = 0
batch_ID = 0
num_frames_ref = len(np.where(batch_id_os11==batch_ID)[0])
sourceid_ref = 1 # choose the star
V3PA = 0         # roll angle
exptime = 30     
cgisim_obj.scene['schedule']['batches'][0] = {'num_timesteps':num_frames_ref,
                                                 'batch_ID':batch_ID,
                                                 'sourceid':sourceid_ref,
                                                 'exptime':exptime,
                                                 'V3PA':V3PA}
cgisim_obj.scene['schedule']['schedule_index_array'].append(np.ones(num_frames_ref)*index_batch_ref)

In [8]:
# Science observation roll 1
index_batch_roll1 = 1
batch_ID = 100
num_frames_roll1 = len(np.where(batch_id_os11==batch_ID)[0])
sourceid_sci = 0    # choose source star
V3PA_roll1 = 13     # roll angle
exptime = 30
cgisim_obj.scene['schedule']['batches'].append({'num_timesteps':num_frames_roll1,
                                                 'batch_ID':batch_ID,
                                                 'sourceid':sourceid_sci,
                                                 'exptime':exptime,
                                                 'V3PA':V3PA_roll1})
cgisim_obj.scene['schedule']['schedule_index_array'].append(np.ones(num_frames_ref)*index_batch_roll1)


In [9]:
# Science observation roll 2
index_batch_roll2 = 2
batch_ID = 101
# num_frames_roll2 = 3
num_frames_roll2 = len(np.where(batch_id_os11==batch_ID)[0])
sourceid_sci = 0 # what star?
V3PA_roll2 = -13 #roll angle
exptime = 30
cgisim_obj.scene['schedule']['batches'].append({'num_timesteps':num_frames_roll2,
                                                 'batch_ID':batch_ID,
                                                 'sourceid':sourceid_sci,
                                                 'exptime':exptime,
                                                 'V3PA':V3PA_roll2})
cgisim_obj.scene['schedule']['schedule_index_array'].append(np.ones(num_frames_ref)*index_batch_roll2)


In [10]:
xoffsets = [50, 100, 200, 250, 100, 200]
yoffsets = [50, 100, 200, 250, -100, -200]

In [11]:
# Add point source companion
cgisim_obj.add_point_source_to_scene(sourceid=2, central_sourceid=0, xoffset=xoffsets[0], yoffset=yoffsets[0])